In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer


In [ ]:
Dataset Directory Path
# -----------------------------
dataset_dir = "path_to_dataset"  # 👈 Change this to your dataset path
# The folder should have subfolders: e.g., dataset/class1, dataset/class2, etc.


In [ ]:
# Parameters
# -----------------------------
IMG_HEIGHT, IMG_WIDTH, CHANNELS = 174, 208, 3
BATCH_SIZE = 32
EPOCHS = 100
NUM_FOLDS = 5


In [ ]:
Data Generator
# -----------------------------
datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.0)

all_data = []
all_labels = []

# Load dataset manually
for class_name in os.listdir(dataset_dir):
    class_path = os.path.join(dataset_dir, class_name)
    if os.path.isdir(class_path):
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = tf.keras.utils.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = tf.keras.utils.img_to_array(img)
            all_data.append(img_array)
            all_labels.append(class_name)

all_data = np.array(all_data, dtype="float32") / 255.0
all_labels = np.array(all_labels)

# One-hot encoding labels
lb = LabelBinarizer()
all_labels = lb.fit_transform(all_labels)

In [ ]:
oft Attention Block
# -----------------------------
def soft_attention_block(inputs):
    attention = layers.Conv2D(1, (1, 1), activation='sigmoid')(inputs)
    return layers.multiply([inputs, attention])

In [ ]:
NN Model with Soft Attention
# -----------------------------
def create_cnn_with_attention(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # Apply Soft Attention
    x = soft_attention_block(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, activation="relu")(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
Custom Callback - Every 10 Epochs
# -----------------------------
class EveryNEpoch(tf.keras.callbacks.Callback):
    def __init__(self, save_path, interval=10):
        super(EveryNEpoch, self).__init__()
        self.interval = interval
        self.save_path = save_path

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.interval == 0:
            acc = logs.get('accuracy')
            val_acc = logs.get('val_accuracy')
            print(f"\n🔔 Epoch {epoch+1}: saving model | Train Acc={acc:.4f}, Val Acc={val_acc:.4f}")
            self.model.save(self.save_path.format(epoch=epoch+1))


In [ ]:
5-Fold Cross Validation
# -----------------------------
kfold = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=42)
fold_no = 1

for train_idx, val_idx in kfold.split(all_data, all_labels):
    print(f"\n📂 Training fold {fold_no}/{NUM_FOLDS} ...")

    X_train, X_val = all_data[train_idx], all_data[val_idx]
    y_train, y_val = all_labels[train_idx], all_labels[val_idx]

    model = create_cnn_with_attention((IMG_HEIGHT, IMG_WIDTH, CHANNELS), num_classes=all_labels.shape[1])

In [ ]:
# Callback: save model every 10 epochs
    checkpoint_path = f"model_fold{fold_no}_epoch{{epoch}}.h5"
    every10 = EveryNEpoch(checkpoint_path, interval=10)

    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        verbose=1,
        callbacks=[every10]
    )

    # Evaluate fold
    scores = model.evaluate(X_val, y_val, verbose=0)
    print(f"✅ Fold {fold_no} - Accuracy: {scores[1]*100:.2f}%")

    fold_no += 1